In [2]:
import os
import sys
ROOT_DIR = os.path.dirname(os.path.abspath('..'))
sys.path.insert(0, os.path.abspath(ROOT_DIR))

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style()
import seaborn as sns
import statsmodels
import sklearn

In [5]:
df_gdp = pd.read_csv('../data/processed/gdp_per_capita.csv')
df_gdp.head()

,Country,Subject,Measure,Year,Value
0,Australia,Gross Domestic Product (GDP); millions,"National currency, current prices",1970,40314.0
1,Australia,Gross Domestic Product (GDP); millions,"National currency, current prices",1971,44484.0
2,Australia,Gross Domestic Product (GDP); millions,"National currency, current prices",1972,49747.0
3,Australia,Gross Domestic Product (GDP); millions,"National currency, current prices",1973,60272.0
4,Australia,Gross Domestic Product (GDP); millions,"National currency, current prices",1974,71158.0


In [7]:
df_prod = pd.read_csv('../data/processed/productivity_growth.csv')
df_prod.head()

,Country,Subject,Measure,Year,Unit,Value
0,Australia,Total capital services,Annual growth/change,1985,Percentage,5.122257
1,Australia,Total capital services,Annual growth/change,1986,Percentage,4.945708
2,Australia,Total capital services,Annual growth/change,1987,Percentage,4.964987
3,Australia,Total capital services,Annual growth/change,1988,Percentage,5.063412
4,Australia,Total capital services,Annual growth/change,1989,Percentage,4.829607


In [8]:
df_gdp['Subject'].unique()

array(['Gross Domestic Product (GDP); millions',
       'GDP per head of population', 'GDP per hour worked',
       'GDP per person employed',
       'Labour utilisation (hours worked per head of population)',
       'Hours worked for total employment; millions',
       'Gap in GDP per hour worked with respect to the USA',
       'Total population; persons; thousands',
       'Gap in GDP per capita with respect to the USA',
       'Total employment (number of persons employed); thousands',
       'Gap in labour utilisation with respect to the USA',
       'Average hours worked per person employed'], dtype=object)

In [10]:
df_gdp['Country'].unique()

array(['Australia', 'Austria', 'Belgium', 'Canada', 'Czech Republic',
       'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Hungary',
       'Iceland', 'Ireland', 'Italy', 'Japan', 'Korea', 'Luxembourg',
       'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Slovak Republic', 'Spain', 'Sweden', 'Switzerland',
       'Turkey', 'United Kingdom', 'United States', 'Brazil', 'Chile',
       "China (People's Republic of)", 'Estonia', 'India', 'Indonesia',
       'Israel', 'Russia', 'Slovenia', 'South Africa',
       'BRIICS economies - Brazil, Russia, India, Indonesia, China and South Africa',
       'European Union (28 countries)', 'OECD - Total', 'G7', 'Colombia',
       'Latvia', 'Lithuania', 'Euro area (19 countries)'], dtype=object)

In [13]:
df_gdp['Measure'].unique()

array(['National currency, current prices',
       'USD, current prices, current PPPs',
       'USD, constant prices, 2015 PPPs', 'As % of the USA (USA=100)',
       'Persons/Hours', 'Gap in % points'], dtype=object)

In [36]:
def make_prod_gdp_data(df_prod, df_gdp):
    mutual_countries = [c for c in df_gdp['Country'].unique() if c in df_prod['Country'].unique()]
    qry = " | ".join([f"(Country == '{i}')" for i in mutual_countries])
    dff_prod = df_prod.query(qry, engine='python')
    dff_gdp = df_gdp.query(qry, engine='python')
    dff_gdp = dff_gdp[dff_gdp['Measure'] == 'USD, current prices, current PPPs']
    
    return(dff_prod, dff_gdp)

dff_prod, dff_gdp = make_prod_gdp_data(df_prod, df_gdp)


In [37]:
dff_prod

,Country,Subject,Measure,Year,Unit,Value
0,Australia,Total capital services,Annual growth/change,1985,Percentage,5.122257
1,Australia,Total capital services,Annual growth/change,1986,Percentage,4.945708
2,Australia,Total capital services,Annual growth/change,1987,Percentage,4.964987
3,Australia,Total capital services,Annual growth/change,1988,Percentage,5.063412
4,Australia,Total capital services,Annual growth/change,1989,Percentage,4.829607
5,Australia,Total capital services,Annual growth/change,1990,Percentage,3.322821
6,Australia,Total capital services,Annual growth/change,1991,Percentage,2.668921
7,Australia,Total capital services,Annual growth/change,1992,Percentage,3.043606
8,Australia,Total capital services,Annual growth/change,1993,Percentage,3.037751
9,Australia,Total capital services,Annual growth/change,1994,Percentage,3.623185


In [12]:
df_gdp[(df_gdp['Subject'] == 'GDP per person employed')& (df_gdp['Country'] == 'Euro area (19 countries)')].sort_values(by='Year')

,Country,Subject,Measure,Year,Value
45773,Euro area (19 countries),GDP per person employed,"National currency, current prices",1995,43848.192452
45973,Euro area (19 countries),GDP per person employed,"USD, constant prices, 2015 PPPs",1995,79325.351767
46073,Euro area (19 countries),GDP per person employed,As % of the USA (USA=100),1995,82.464083
45873,Euro area (19 countries),GDP per person employed,"USD, current prices, current PPPs",1995,49433.861049
46074,Euro area (19 countries),GDP per person employed,As % of the USA (USA=100),1996,81.209045
45874,Euro area (19 countries),GDP per person employed,"USD, current prices, current PPPs",1996,50761.922130
45974,Euro area (19 countries),GDP per person employed,"USD, constant prices, 2015 PPPs",1996,80096.659776
45774,Euro area (19 countries),GDP per person employed,"National currency, current prices",1996,45422.162796
46075,Euro area (19 countries),GDP per person employed,As % of the USA (USA=100),1997,80.696258
45775,Euro area (19 countries),GDP per person employed,"National currency, current prices",1997,46062.768704
